<a href="https://colab.research.google.com/github/Diwakarmogarala/Capstone/blob/main/Phase_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://drive.google.com/drive/u/0/folders/11zhBUqnUT6J4IVMLpBcrgz8saXAGJ7Hi

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import polars as pl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pl.read_csv("/content/drive/MyDrive/Capstone Project/merged_data_with_prices.csv")

In [ ]:
# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# encoded = le.fit_transform(data["Age Group"].to_numpy())
# data = data.with_columns(
#     pl.Series("Age_Group", encoded, dtype=pl.Int32)
# )


In [ ]:
data.columns

['Hospital County',
 'Age Group',
 'Gender',
 'Race',
 'Ethnicity',
 'Type of Admission',
 'Patient Disposition',
 'APR DRG Description',
 'APR MDC Description',
 'APR Severity of Illness Description',
 'APR Risk of Mortality',
 'APR Medical Surgical Description',
 'Payment Typology 1',
 'Payment Typology 2',
 'Payment Typology 3',
 'Length of Stay',
 'Age_Group',
 'Total Charges',
 'Total Costs',
 'Age',
 'Year']

In [ ]:
data["Age_Group"].value_counts()

Age_Group,count
i64,u32
3,4376833
2,3033309
1,1498033
4,4549300
0,2212893


In [ ]:
data.head()

Hospital County,Age Group,Gender,Race,Ethnicity,Type of Admission,Patient Disposition,APR DRG Description,APR MDC Description,APR Severity of Illness Description,APR Risk of Mortality,APR Medical Surgical Description,Payment Typology 1,Payment Typology 2,Payment Typology 3,Length of Stay,Age_Group,Total Charges,Total Costs,Age,Year
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,i64,f64,f64,i64,i64
"""Allegany""","""70 or Older""","""F""","""White""","""Not Span/Hispanic""","""Urgent""","""Home or Self Care""","""other pneumonia""","""diseases and disorders of the …","""Moderate""","""Moderate""","""Medical""","""Medicare""","""Private Health Insurance""","""None""",3,4,3913.23,3466.83,4,2016
"""Allegany""","""30 to 49""","""M""","""White""","""Not Span/Hispanic""","""Elective""","""Home or Self Care""","""other pneumonia""","""diseases and disorders of the …","""Minor""","""Minor""","""Medical""","""Self-Pay""","""None""","""None""",2,2,3597.1,3746.8,2,2016
"""Allegany""","""50 to 69""","""F""","""White""","""Not Span/Hispanic""","""Urgent""","""Home or Self Care""","""chronic obstructive pulmonary …","""diseases and disorders of the …","""Minor""","""Minor""","""Medical""","""Medicare""","""Federal/State/Local/VA""","""None""",7,3,9149.22,8322.1,3,2016
"""Allegany""","""70 or Older""","""M""","""White""","""Not Span/Hispanic""","""Urgent""","""Home or Self Care""","""other pneumonia""","""diseases and disorders of the …","""Moderate""","""Moderate""","""Medical""","""Medicare""","""Medicaid""","""None""",4,4,5880.61,5828.87,4,2016
"""Allegany""","""50 to 69""","""M""","""White""","""Not Span/Hispanic""","""Elective""","""Home or Self Care""","""other pneumonia""","""diseases and disorders of the …","""Moderate""","""Moderate""","""Medical""","""Blue Cross/Blue Shield""","""None""","""None""",5,3,7019.76,6604.0,3,2016


In [ ]:
np.corrcoef(
        data["Total Charges"].to_numpy(),
        data["Age_Group"].to_numpy()
    )[0, 1]

np.float64(0.1117900031586466)

In [ ]:
data.columns

['Hospital County',
 'Age Group',
 'Gender',
 'Race',
 'Ethnicity',
 'Type of Admission',
 'Patient Disposition',
 'APR DRG Description',
 'APR MDC Description',
 'APR Severity of Illness Description',
 'APR Risk of Mortality',
 'APR Medical Surgical Description',
 'Payment Typology 1',
 'Payment Typology 2',
 'Payment Typology 3',
 'Length of Stay',
 'Age_Group',
 'Total Charges',
 'Total Costs',
 'Age',
 'Year']

In [ ]:
X = data[['Year',
          'Hospital County',
          'Age Group',
          'Gender',
          'Race',
          'Ethnicity',
          'Type of Admission',
          'Patient Disposition',
          'APR DRG Description',
          'APR MDC Description',
          'APR Severity of Illness Description',
          'APR Risk of Mortality',
          'APR Medical Surgical Description',
          'Payment Typology 1',
          'Payment Typology 2',
          'Payment Typology 3'
          ]]
y = data["Length of Stay"]

In [ ]:
X.dtypes

[Int64,
 String,
 String,
 String,
 String,
 String,
 String,
 String,
 String,
 String,
 String,
 String,
 String,
 String,
 String,
 String]

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for i in range(len(X.columns)):
    if X.dtypes[i] == X.dtypes[2]:
        encoded = le.fit_transform(X[X.columns[i]].to_numpy())
        X = X.with_columns(
            pl.Series(X.columns[i], encoded, dtype=pl.Int32)
            )


In [ ]:
from sklearn.preprocessing import LabelEncoder

cat_cols = [c for c, dt in zip(X.columns, X.dtypes) if dt == pl.Utf8]

encoded_series = []
for c in cat_cols:
    le = LabelEncoder()                                  # new encoder each col
    codes = le.fit_transform(X[c].to_numpy())            # NumPy 1-D
    encoded_series.append(pl.Series(c, codes, pl.Int32))

X = X.with_columns(encoded_series)


In [ ]:
# Export X (Polars DataFrame)
X.write_csv("X.csv")

# Wrap y (Polars Series) into a DataFrame before saving
pl.DataFrame({"Total Charges": y}).write_csv("y.csv")


ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/polars/dataframe/frame.py", line 775, in shape
    return self._df.shape()
           ^^^^^^^^^^^^^^^^
RuntimeError: Already mutably borrowed
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/polars/dataframe/frame.py", line 775, in shape
    return self._df.shape()
           ^^^^^^^^^^^^^^^^
RuntimeError: Already mutably borrowed
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call

In [ ]:
X.to_csv("/content/drive/MyDrive/Capstone Project/X.csv", index=False)
y.to_csv("/content/drive/MyDrive/Capstone Project/y.csv", index=False)

AttributeError: 'DataFrame' object has no attribute 'to_csv'

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42, stratify = X["APR DRG Description"])

In [ ]:
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
import joblib
rf = RandomForestClassifier(
        n_estimators=0,          # start with no trees
        warm_start=True,         # lets us add trees later
        max_depth=None,
        n_jobs=-1,
        random_state=42,
        max_features="sqrt",     # speed, no big loss in perf
)

trees_per_chunk = 25
total_trees     = 200            # tune later
for step in range(0, total_trees, trees_per_chunk):
    rf.n_estimators += trees_per_chunk
    rf.fit(X_train, y_train)     # reuse the full data
    joblib.dump(rf, "/content/drive/MyDrive/rf_ckpt.pkl")   # ✅ survives disconnect
    print(f"{rf.n_estimators} trees built")

# reload after reconnect:
# rf = joblib.load("/content/drive/MyDrive/rf_ckpt.pkl")


In [ ]:
!pip install -q rapidsai-cu12 pylibraft-cu12 --extra-index-url=https://pypi.nvidia.com
import cudf, cuml

ERROR: Could not find a version that satisfies the requirement rapidsai-cu12 (from versions: none)
ERROR: No matching distribution found for rapidsai-cu12


In [ ]:
import cudf

# X_train is Polars, already numeric / label-encoded
Xc = cudf.DataFrame.from_arrow(X_train.to_arrow())   # zero-copy to GPU
yc = cudf.Series(y_train.to_numpy())                 # 1-D target

from cuml.ensemble import RandomForestClassifier

rf_gpu = RandomForestClassifier(
             n_estimators=400,
             max_depth=25,
             n_streams=8,        # “threads” on the GPU
)
rf_gpu.fit(Xc, yc)

pred = rf_gpu.predict(cudf.DataFrame.from_arrow(X_test.to_arrow()))


In [ ]:
# prompt: know my cores

import multiprocessing

num_cores = multiprocessing.cpu_count()
print(f"Number of CPU cores available: {num_cores}")

Number of CPU cores available: 8


In [ ]:
from transformers import AutoTokenizer

MODEL_NAME = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext"
tok = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(x_train, y_train)


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [ ]:
import dask

In [ ]:
from dask.distributed import Client

client = Client(n_workers=8, threads_per_worker=2, memory_limit="45GB")
client


INFO:distributed.http.proxy:To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:43931
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:8787/status
INFO:distributed.scheduler:Registering Worker plugin shuffle
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:36187'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:36143'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:42127'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:33183'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:32845'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:37067'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:36351'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:43819'
INFO:distributed.scheduler:Register wor

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 16,Total memory: 335.28 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43931,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 335.28 GiB
Comm: tcp://127.0.0.1:38505,Total threads: 2
Dashboard: http://127.0.0.1:36697/status,Memory: 41.91 GiB
Nanny: tcp://127.0.0.1:36187,


In [9]:
# https://drive.google.com/drive/u/0/folders/11zhBUqnUT6J4IVMLpBcrgz8saXAGJ7Hi

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from xgboost import XGBRegressor

In [6]:
# https://drive.google.com/drive/u/0/folders/11zhBUqnUT6J4IVMLpBcrgz8saXAGJ7Hi
import joblib
from google.colab import drive
drive.mount('/content/drive')
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import pandas as pl
X = pl.read_csv("/content/drive/MyDrive/Capstone Project/X.csv")
y = pl.read_csv("/content/drive/MyDrive/Capstone Project/y.csv")
x_train, x_test, y_train, y_test = train_test_split(X[X["Year"]==2016], y[X["Year"]==2016], test_size=0.5, random_state=42,stratify=X[X["Year"]==2016]['APR DRG Description'])
model = XGBRegressor()
model.fit(x_train, y_train)

from sklearn.metrics import mean_squared_error, r2_score
y_pred = model.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
# prompt: export model as pkl using joblib to my drive Capstone Project

joblib.dump(model, "/content/drive/MyDrive/Capstone Project/XGB_regressor_2016.pkl")
r2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


0.41969311237335205

In [7]:
x_train, x_test, y_train, y_test = train_test_split(X[X["Year"]==2017], y[X["Year"]==2017], test_size=0.5, random_state=42, stratify=X[X["Year"]==2017]['APR DRG Description'])
model = XGBRegressor()
model.fit(x_train, y_train)

from sklearn.metrics import mean_squared_error, r2_score
y_pred = model.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
# prompt: export model as pkl using joblib to my drive Capstone Project

joblib.dump(model, "/content/drive/MyDrive/Capstone Project/XGB_regressor_2017.pkl")
r2

0.4256724715232849

In [8]:
x_train, x_test, y_train, y_test = train_test_split(X[X["Year"]==2018], y[X["Year"]==2018], test_size=0.5, random_state=42, stratify=X[X["Year"]==2018]['APR DRG Description'])
model = XGBRegressor()
model.fit(x_train, y_train)

from sklearn.metrics import mean_squared_error, r2_score
y_pred = model.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
# prompt: export model as pkl using joblib to my drive Capstone Project

joblib.dump(model, "/content/drive/MyDrive/Capstone Project/XGB_regressor_2018.pkl")
r2

0.42332279682159424

In [9]:
x_train, x_test, y_train, y_test = train_test_split(X[X["Year"]==2019], y[X["Year"]==2019], test_size=0.5, random_state=42, stratify=X[X["Year"]==2019]['APR DRG Description'])
model = XGBRegressor()
model.fit(x_train, y_train)

from sklearn.metrics import mean_squared_error, r2_score
y_pred = model.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
# prompt: export model as pkl using joblib to my drive Capstone Project

joblib.dump(model, "/content/drive/MyDrive/Capstone Project/XGB_regressor_2019.pkl")
r2

0.4203885793685913

In [10]:
x_train, x_test, y_train, y_test = train_test_split(X[X["Year"]==2020], y[X["Year"]==2020], test_size=0.5, random_state=42, stratify=X[X["Year"]==2020]['APR DRG Description'])
model = XGBRegressor()
model.fit(x_train, y_train)

from sklearn.metrics import mean_squared_error, r2_score
y_pred = model.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
# prompt: export model as pkl using joblib to my drive Capstone Project

joblib.dump(model, "/content/drive/MyDrive/Capstone Project/XGB_regressor_2020.pkl")
r2

0.4422503709793091

In [11]:
x_train, x_test, y_train, y_test = train_test_split(X[X["Year"]==2021], y[X["Year"]==2021], test_size=0.5, random_state=42, stratify=X[X["Year"]==2021]['APR DRG Description'])
model = XGBRegressor()
model.fit(x_train, y_train)

from sklearn.metrics import mean_squared_error, r2_score
y_pred = model.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
# prompt: export model as pkl using joblib to my drive Capstone Project

joblib.dump(model, "/content/drive/MyDrive/Capstone Project/XGB_regressor_2021.pkl")
r2

0.461844801902771

In [12]:
x_train, x_test, y_train, y_test = train_test_split(X[X["Year"]==2022], y[X["Year"]==2022], test_size=0.5, random_state=42, stratify=X[X["Year"]==2022]['APR DRG Description'])
model = XGBRegressor()
model.fit(x_train, y_train)

from sklearn.metrics import mean_squared_error, r2_score
y_pred = model.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
# prompt: export model as pkl using joblib to my drive Capstone Project

joblib.dump(model, "/content/drive/MyDrive/Capstone Project/XGB_regressor_2022.pkl")
r2

0.4254606366157532

In [1]:
# https://drive.google.com/drive/u/0/folders/11zhBUqnUT6J4IVMLpBcrgz8saXAGJ7Hi
import joblib
from google.colab import drive
drive.mount('/content/drive')
from sklearn.model_selection import train_test_split
import pandas as pl
X = pl.read_csv("/content/drive/MyDrive/Capstone Project/X.csv")
y = pl.read_csv("/content/drive/MyDrive/Capstone Project/y.csv")
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42, stratify=X['APR DRG Description'])


Mounted at /content/drive


NameError: name 'XGBRegressor' is not defined

In [3]:
from xgboost import XGBRegressor
model = XGBRegressor()
model.fit(x_train, y_train)

from sklearn.metrics import mean_squared_error, r2_score
y_pred = model.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
# prompt: export model as pkl using joblib to my drive Capstone Project

joblib.dump(model, "/content/drive/MyDrive/Capstone Project/XGB_regressor_2016_2022.pkl")
r2

0.421078085899353

In [6]:
from sklearn.metrics import mean_squared_error, r2_score
mean_squared_error(y_test, y_pred)


39.324485778808594

In [10]:
from catboost import CatBoostRegressor
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42, stratify=y)
model = CatBoostRegressor()
model.fit(x_train, y_train)

from sklearn.metrics import mean_squared_error, r2_score
y_pred = model.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
# prompt: export model as pkl using joblib to my drive Capstone Project

joblib.dump(model, "/content/drive/MyDrive/Capstone Project/XGB_regressor_2016_2022.pkl")
r2

Learning rate set to 0.168826
0:	learn: 7.9281167	total: 356ms	remaining: 5m 56s
1:	learn: 7.6977610	total: 619ms	remaining: 5m 9s
2:	learn: 7.5289268	total: 886ms	remaining: 4m 54s
3:	learn: 7.3950059	total: 1.16s	remaining: 4m 49s
4:	learn: 7.2850467	total: 1.45s	remaining: 4m 48s
5:	learn: 7.2119394	total: 1.71s	remaining: 4m 43s
6:	learn: 7.1392155	total: 1.97s	remaining: 4m 39s
7:	learn: 7.0809061	total: 2.27s	remaining: 4m 40s
8:	learn: 7.0370918	total: 2.53s	remaining: 4m 38s
9:	learn: 7.0023018	total: 2.8s	remaining: 4m 37s
10:	learn: 6.9713825	total: 3.05s	remaining: 4m 34s
11:	learn: 6.9315726	total: 3.32s	remaining: 4m 33s
12:	learn: 6.9079504	total: 3.58s	remaining: 4m 32s
13:	learn: 6.8882598	total: 3.84s	remaining: 4m 30s
14:	learn: 6.8711355	total: 4.11s	remaining: 4m 29s
15:	learn: 6.8538448	total: 4.39s	remaining: 4m 30s
16:	learn: 6.8372173	total: 4.68s	remaining: 4m 30s
17:	learn: 6.8255273	total: 4.96s	remaining: 4m 30s
18:	learn: 6.8112947	total: 5.21s	remaining: 4

0.4367115035054613

In [11]:
mse

38.238963755701924

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 25.7 MB/s eta 0:00:00
